In [11]:
import random  
import math
MIN_MUTATION_RATE=0.01
MAX_DISTANCE_SUM=2000
TSP_SIZE=9

# Below are the distances between each of the ten locations that I plan to visit on my trip.
# Location A is the first column and row, location B is the second column and row, etc. For example, 25km is the 
# distance between Location A and Location B, and 43km is the distance between Location A and Location C.
mat = [[ 0, 25, 43, 57, 43, 61, 29, 41, 48, 71], 
       [ 25, 0, 29, 34, 43, 68, 49, 66, 72, 91], 
       [ 43, 29, 0, 52, 72, 96, 72, 81, 89,114], 
       [ 57, 34, 52, 0, 45, 71, 71, 95, 99,108], 
       [ 43, 43, 72, 45, 0, 27, 36, 65, 65, 65], 
       [ 61, 68, 96, 71, 27, 0, 40, 66, 62, 46], 
       [ 29, 49, 72, 71, 36, 40, 0, 31, 31, 43], 
       [ 41, 66, 81, 95, 65, 66, 31, 0, 11, 46], 
       [ 48, 72, 89, 99, 65, 62, 31, 11, 0, 36], 
       [ 71, 91,114,108, 65, 46, 43, 46, 36, 0]]

# Calculating for the shortest path distance for a 'chromosome' in the genetic algorithm, or a travelling route in 
# this case.
def TSP_distance (chromo):    
    distance=mat[0][chromo[0]]
    loc=0    
    for gene in chromo:
        distance = distance + mat[chromo[loc]][chromo[loc+1]]
        loc=loc+1
        if(loc==len(chromo)-1):
            return(distance)

# As the shorter the route is, the better, we subtract the TSP_distance from MAX_DISTANCE_SUM to attribute a greater
# number to the better (which is the shorter) routes. The fitness function, fitnessftn, is a measure of how short the
# route is.
def fitnessftn (chromo):    
    return(MAX_DISTANCE_SUM-TSP_distance(chromo))

# Generating random travelling routes.
def ran_seq(num): 
    alist=[]
    for i in range(0,num):               
        alist.append(new_rand_ele(num,alist))
    return(alist)

# Generating random numbers (or, locations to visit next) to append to the ran_seq above.
def new_rand_ele(num, alist):
    while 1:
        gennum = int(random.random()*num+1)
        if(ele_in_alist(gennum, alist)==0):
            return(gennum)

def ele_in_alist(num, alist):
    for i in alist:
        if(num==i):
            return 1
    return 0

# Generating an initial population of routes, to then apply the genetic algorithm to.
def init_population(num, size):
    alist=[]
    for i in range (0,size):
        blist=ran_seq(num)
        alist.append(blist)
    return(alist)

# Selecting random routes from the population for 'recombination' to produce 'offsprings'.
# The greater the fitness function, the greater chance a route has of being selected. This is an imitation of the
# survival of the fittest.
def selection(population):
    alist=[]
    totfit=0
    for chromo in population:
        fit=fitnessftn(chromo)
        totfit=totfit+fit
        alist.append(fit)
    randnum=random.random() # generating a random number, randnum
    index=0
    fitsum=0
    for fit in alist:
        fitsum=fitsum+fit       
        if(randnum <fitsum/totfit):
            break
        index=index+1
    return(population[index])
    
# We use a method called uniform order-based crossover to generate 'child' routes from two 'parent' 
# routes to generate a route with hopefully a greater fitness function.
def uniform_order_based_crossover_with_mutation(paren1, paren2, rate):
    template=[]
    resul=[]
    for i in range(0,len(paren1)):
        template.append(int(random.random()*2))
    index=0
    child1=[]
    child2=[]
    rem1=[]
    rem2=[]
    for i in template:
        if(i==1):
            child1.append(paren1[index])
            child2.append(paren2[index])            
        else:
            child1.append(0)
            child2.append(0)
            rem1.append(paren1[index])
            rem2.append(paren2[index])
        index=index+1
    sublist1=sorted_sublist(rem1,paren2)
    sublist2=sorted_sublist(rem2,paren1)
    remindex=0
    index=0
    for i in child1:
        if(i==0):
            child1[index]=sublist1[remindex]
            child2[index]=sublist2[remindex]
            remindex=remindex+1
        index=index+1
    resul.append(per_mutation(child1,rate))
    resul.append(per_mutation(child2,rate))
    return(resul)

def sorted_sublist(small,large):
    alist=[]
    for i in large:
        if(ele_in_alist(i, small)==1):
            alist.append(i)
    return(alist)

def GA_TSP(pop_size,num_gen, mutation_rate, elite_num): # Genetic Algorithm _ The Shortest Path
    # Population size (the number of routes to produce every generation), the number of generatons, the mutation
    # rate, and the number of 'elite chromosomes' to pass on (a certain number of the shortest routes are 
    # automatically passed onto the next generation)
    old_gen=init_population(TSP_SIZE,pop_size)  
    best_route=best_chro(old_gen)  # Finding the best 'chromosome' and attributing it to best_route.
    print("Avg Fit of Gen 0", total_fitness(old_gen)/pop_size)    # Finding the average fitness score of the 
    # first 'generation' of routes generated.
    print("Gen 0 Best Route Fitness",fitnessftn(best_route),best_route) #Finding the best fitness score out of the
    # first 'generation' of routes generated.
    crossoverpairs_num= int((pop_size-elite_num)/2)  
    best_avg_fit=total_fitness(old_gen)/pop_size
    for j in range(0,num_gen):                
        new_gen=[]
        elites=[]
        bbb=elite_chros_indices(old_gen,elite_num)        
        for i in bbb:
            new_gen.append(old_gen[i])
        for i in range(0,crossoverpairs_num):        
            crossovered=uniform_order_based_crossover_with_mutation(selection(old_gen),selection(old_gen),mutation_rate)
            new_gen.extend(crossovered)
        best_route_new_gen=best_chro(new_gen)
        if(fitnessftn(best_route_new_gen)>fitnessftn(best_route)):
            best_route=best_route_new_gen
            print("Gen", j+1,"Best Route Fitness",fitnessftn(best_route),best_route)
        avg_fit=total_fitness(new_gen)/pop_size          
        if(avg_fit>best_avg_fit):
            best_avg_fit=avg_fit
            print("Avg Fit of Gen ", j+1, "=",avg_fit) # print message everytime the best route fitness is improved
            # across generations.
        old_gen=new_gen
    print("Gen", j+1,"Best Route Distance",TSP_distance(best_route),best_route)
    
# Calculating for the total fitness score of a generation of routes.
def total_fitness(population):
    totfit=0
    for chromo in population:
        fit=fitnessftn(chromo)
        totfit=totfit+fit
    return(totfit)

# Finding the best fitness score from a generation of routes.
def best_chro(population):
    MINFIT=-1000
    best_index=0
    index=0    
    best_fit=MINFIT
    for chromo in population:
        fit=fitnessftn(chromo)
        if(best_fit <fit):
            best_index=index
            best_fit=fit
        index=index+1        
    return(population[best_index])

# Finding the index of the best chromosome in a population.
def best_chro_index(alist,exceptlist):
    MINFIT=-1000
    best_index=0
    index=0    
    best_fit=MINFIT
    for chromo in alist:
        if(element_in_list(index,exceptlist)==1):
            index=index+1
            continue
        fit=fitnessftn(chromo)
        if(best_fit <fit):
            best_index=index
            best_fit=fit
        index=index+1        
    return(best_index)

# Collecting the indicies of the best chromosomes, or the shortest routes, produced in a generation.
def elite_chros_indices(alist,num):  
    resul=[]    
    for i in range(0,num):
        best_index=best_chro_index(alist,resul)
        resul.append(best_index)
    return(resul)

def element_in_list(ele,alist):
    for i in alist:
        if(ele==i):
            return(1)
    return(0)

def per_mutation(alist, rate):
    index=0    
    listlen=len(alist)
    firstele=alist[0]
    secondele=alist[1]
    blist=[]
    for i in alist:    
        if(index<listlen-1):
            if(random.random()<rate):
                blist.append(secondele)
            else:
                blist.append(firstele)
                firstele=secondele
        else:
            break
        index=index+1
        if(index<listlen-1):
            secondele=alist[index+1]
        else:
            break
    if(random.random()<rate):
        blist.append(blist[0])        
        blist[0]=firstele
    else:
        blist.append(firstele)        
    return(blist)

In [42]:
ran_seq(5)
# Testing the ran_seq function.

[1, 4, 2, 3, 5]

In [43]:
mat

[[0, 25, 43, 57, 43, 61, 29, 41, 48, 71],
 [25, 0, 29, 34, 43, 68, 49, 66, 72, 91],
 [43, 29, 0, 52, 72, 96, 72, 81, 89, 114],
 [57, 34, 52, 0, 45, 71, 71, 95, 99, 108],
 [43, 43, 72, 45, 0, 27, 36, 65, 65, 65],
 [61, 68, 96, 71, 27, 0, 40, 66, 62, 46],
 [29, 49, 72, 71, 36, 40, 0, 31, 31, 43],
 [41, 66, 81, 95, 65, 66, 31, 0, 11, 46],
 [48, 72, 89, 99, 65, 62, 31, 11, 0, 36],
 [71, 91, 114, 108, 65, 46, 43, 46, 36, 0]]

In [44]:
chromosome1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
chromosome2 = [1, 2, 6, 7, 8, 9, 3, 4, 5]
chromosome3 = [9, 3, 4, 5, 1, 2, 6, 7, 8]
# Test with three random 'chromosomes'. This is how a route would be formatted. 
# For example, chromosome3 is a route that starts from Location 9, then goes to 3, then goes to 4, etc.

In [26]:
print(fitnessftn(chromosome1))
print(fitnessftn(chromosome2))
print(fitnessftn(chromosome3))
# These are the total distances that the travel would take using each of the routes. It can be seen that chromosome3
# is the shortest route.

1704
1616
1538


In [24]:
ipop=init_population(9,40)
# Testing the init_population function to generate a population of routes.

In [33]:
ipop

[[8, 2, 7, 1, 3, 9, 4, 5, 6],
 [5, 6, 9, 7, 8, 1, 2, 3, 4],
 [6, 8, 5, 9, 4, 1, 3, 7, 2],
 [4, 2, 8, 3, 7, 6, 1, 9, 5],
 [4, 8, 9, 3, 6, 5, 1, 2, 7],
 [4, 8, 9, 6, 7, 3, 1, 5, 2],
 [8, 2, 6, 9, 5, 3, 7, 1, 4],
 [7, 9, 1, 8, 3, 4, 5, 6, 2],
 [6, 1, 4, 3, 9, 8, 7, 2, 5],
 [5, 2, 3, 4, 8, 6, 1, 9, 7],
 [5, 9, 2, 3, 8, 7, 4, 6, 1],
 [8, 5, 1, 2, 3, 6, 4, 9, 7],
 [4, 5, 1, 2, 6, 8, 3, 7, 9],
 [1, 9, 4, 2, 6, 8, 7, 5, 3],
 [7, 8, 6, 9, 2, 4, 5, 3, 1],
 [4, 8, 9, 7, 5, 1, 6, 3, 2],
 [3, 6, 7, 1, 5, 8, 9, 2, 4],
 [9, 2, 1, 4, 6, 8, 5, 3, 7],
 [8, 9, 1, 6, 2, 3, 4, 7, 5],
 [2, 8, 7, 5, 9, 6, 4, 1, 3],
 [5, 3, 6, 7, 2, 8, 4, 9, 1],
 [6, 2, 1, 4, 7, 3, 9, 5, 8],
 [2, 4, 1, 8, 7, 6, 5, 9, 3],
 [3, 7, 8, 4, 2, 5, 9, 1, 6],
 [7, 6, 3, 4, 5, 8, 1, 2, 9],
 [3, 9, 7, 2, 5, 1, 6, 4, 8],
 [5, 3, 9, 4, 8, 1, 6, 7, 2],
 [9, 8, 2, 6, 1, 3, 5, 4, 7],
 [5, 1, 4, 9, 7, 3, 2, 6, 8],
 [7, 4, 9, 3, 5, 1, 8, 6, 2],
 [4, 5, 6, 2, 9, 8, 1, 3, 7],
 [5, 6, 8, 1, 4, 9, 2, 7, 3],
 [8, 3, 4, 2, 5, 1, 7, 6, 9],
 [8, 1, 7,

In [34]:
selection(ipop)
# Selecting a random route from ipop.

[3, 7, 8, 4, 2, 5, 9, 1, 6]

In [40]:
GA_TSP(40,1000, 0.02, 0)
# Finding the shortest path distance using 40 routes generated per generation, 1000 generations, and a mutation rate
# of 0.02 (meaning that there is a 2% chance of a mutation occuring).

Avg Fit of Gen 0 1459.075
Gen 0 Best Route Fitness 1545 [8, 4, 5, 1, 3, 2, 6, 9, 7]
Gen 1 Best Route Fitness 1570 [3, 1, 4, 9, 7, 8, 5, 6, 2]
Avg Fit of Gen  1 = 1459.325
Gen 2 Best Route Fitness 1583 [6, 4, 5, 1, 3, 2, 8, 9, 7]
Gen 3 Best Route Fitness 1647 [1, 2, 3, 5, 4, 6, 8, 9, 7]
Avg Fit of Gen  3 = 1468.475
Avg Fit of Gen  4 = 1476.65
Avg Fit of Gen  10 = 1477.475
Avg Fit of Gen  14 = 1477.55
Avg Fit of Gen  15 = 1479.625
Avg Fit of Gen  17 = 1490.2
Gen 29 Best Route Fitness 1653 [7, 8, 5, 9, 6, 4, 3, 1, 2]
Gen 41 Best Route Fitness 1661 [2, 3, 1, 6, 8, 7, 9, 5, 4]
Avg Fit of Gen  41 = 1506.95
Avg Fit of Gen  89 = 1512.75
Gen 93 Best Route Fitness 1663 [2, 1, 3, 4, 6, 7, 8, 5, 9]
Gen 94 Best Route Fitness 1672 [2, 1, 3, 5, 4, 6, 8, 7, 9]
Gen 95 Best Route Fitness 1694 [2, 1, 3, 4, 5, 6, 8, 7, 9]
Avg Fit of Gen  95 = 1515.5
Avg Fit of Gen  119 = 1519.075
Avg Fit of Gen  143 = 1521.75
Avg Fit of Gen  144 = 1526.625
Gen 246 Best Route Fitness 1698 [1, 2, 3, 4, 5, 9, 8, 7, 6]
Gen 69

In [ ]:
# The best route generated is [6,7,8,9,5,4,3,1,2] according to the above calculations, and is a distance of 288km.
# If we run the program again:

In [41]:
GA_TSP(40,1000, 0.02, 0)

Avg Fit of Gen 0 1463.025
Gen 0 Best Route Fitness 1622 [1, 3, 4, 5, 8, 9, 7, 6, 2]
Avg Fit of Gen  6 = 1463.8
Gen 12 Best Route Fitness 1631 [1, 2, 3, 6, 9, 5, 4, 7, 8]
Avg Fit of Gen  12 = 1482.025
Avg Fit of Gen  14 = 1490.675
Gen 38 Best Route Fitness 1664 [6, 8, 7, 1, 2, 3, 4, 5, 9]
Avg Fit of Gen  101 = 1500.425
Avg Fit of Gen  184 = 1504.075
Gen 203 Best Route Fitness 1670 [2, 3, 1, 4, 5, 9, 6, 7, 8]
Avg Fit of Gen  337 = 1506.35
Avg Fit of Gen  365 = 1510.775
Gen 371 Best Route Fitness 1684 [6, 8, 7, 9, 5, 4, 3, 2, 1]
Avg Fit of Gen  371 = 1519.725
Avg Fit of Gen  422 = 1520.825
Avg Fit of Gen  428 = 1521.475
Avg Fit of Gen  429 = 1538.125
Gen 430 Best Route Fitness 1694 [6, 7, 8, 9, 5, 4, 3, 2, 1]
Avg Fit of Gen  432 = 1541.875
Gen 1000 Best Route Distance 306 [6, 7, 8, 9, 5, 4, 3, 2, 1]


In [46]:
# This time, we get the best route as [6,7,8,9,5,4,3,2,1], and it has a distance of 306km, which is a bit longer than
# the previously attained route.
# The genetic algorithm can never guarantee the shortest possible path; that would only be possible by checking
# every single route possible, which would take an astronomical amount of time. The genetic algorithm, however, uses
# a method inspired by evolution in which it constantly survives the routes with higher fitness functions to 
#'reproduce' and pass on their 'desirable genes' (survival of the fittest) to the next 'generation', along with 
# random mutations occuring along the way to make way for the possibility of a randomly generated, but better, route. 